In [2]:
%load_ext autoreload
%autoreload 2

import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import json
import os
from rolling_horizon_optimisation import run_rolling_horizon_opt

# Set ERCOT load zone and solar capacity
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'

In [2]:
# READ IN DATA
# TODO: update as 'electricity' here is a normalised 8760 solar profile
hourly_dayahead_realtime_solar = pd.read_csv("hourly_dayahead_realtime_solar.csv")

# Filter by load zone and complete data
ERCOT_LOAD_ZONE = 'LZ_HOUSTON'
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar[hourly_dayahead_realtime_solar['zone'] == ERCOT_LOAD_ZONE]
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz[hourly_dayahead_realtime_solar_lz['electricity'].notna()]

number_of_days_test = 100
hourly_dayahead_realtime_solar_lz = hourly_dayahead_realtime_solar_lz.iloc[0:24*number_of_days_test,]
# Sets time duration of the optimisation 
# and local_time used for ERCOT realtime and dayahead prices
T = hourly_dayahead_realtime_solar_lz.shape[0]
print(T)

2400


In [3]:
# FACILITY
FACILITY_LIFETIME = 25
OFFTAKE_TPD = 10
GREEN_THRESHOLD_kg_per_kg = 0.4
NUM_DAYS = math.ceil(T/24)
H2_SALES_PRICE_DOL_per_kg = 4.5

# POWER
#   SOLAR
SOLAR_CAPACITY = 400
SOLAR_PPA_DOL_KWH = 0.03 # 2.5 cents per kWh ($USD 50 per MWh)
hourly_solar_production_kwh = hourly_dayahead_realtime_solar_lz['electricity']/hourly_dayahead_realtime_solar_lz['electricity'].max()
#   GRID 
#       CONVERT $/MWh -> $/kWh
hourly_dayahead_dol_per_kwh = hourly_dayahead_realtime_solar_lz['price_dayahead'] / 1000
ERCTO_CO2_kg_per_kwh = 0.42062
hourly_dayahead_dol_per_kwh =  hourly_dayahead_dol_per_kwh.reset_index(drop=True)
# hourly_dayahead_dol_per_kwh.iloc[20] = 0.3

# hourly_dayahead_dol_per_kwh

# EQUIPMENT
#   OPERATIONAL 
#       EFFICIENCY
ELECTROLYSIS_EFFICIENCY = 55
COMPRESSION_EFFICIENCY = 4
LIQUEFACTION_EFFICIENCY = 10
FUEL_CELL_EFFICIENCY = 20
#       LOWER OPERATION BOUNDS (% of nameplate capacity)
ELECTROLYSIS_OPERATION_LOWER = 0.15
COMPRESSOR_OPERATION_LOWER = 0.15
LIQUEFACTION_OPERATION_LOWER = 0.60 
#       RAMP CONSTRAINTS (% of nameplate capacity)
LIQUEFACTION_MAX_UP = 0.1 # 10% per hour up
LIQUEFACTION_MAX_DOWN = 0.1 # 10% per hour down

#   CAPEX
#   CAPEX - total
ELECTROLYSIS_CAPEX_DOL_TPD = 900000 # 1.9 million $USD per TPD of electrolysis
COMPRESSOR_CAPEX_DOL_TPD = 900000 # 0.8 million $USD per TPD of comrpessor
LIQUEFACTION_CAPEX_DOL_TPD = 1945000 # 2.945 million $USD per TPD of liquefaction per day
FUELCELL_CAPEX_DOL_TPD = 100000 # 1.945 million $USD per TPD of liquefaction per day

#   CAPEX - daily, for fast testing 
ELECTROLYSIS_CAPEX_DOL_kgPD = ELECTROLYSIS_CAPEX_DOL_TPD / 1000
COMPRESSOR_CAPEX_DOL_kgPD = COMPRESSOR_CAPEX_DOL_TPD / 1000
LIQUEFACTION_CAPEX_DOL_kgPD = LIQUEFACTION_CAPEX_DOL_TPD / 1000
FUELCELL_CAPEX_DOL_kgPD = FUELCELL_CAPEX_DOL_TPD / 1000

ELECTROLYSIS_CAPEX_DOL_kgPD = NUM_DAYS*ELECTROLYSIS_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
COMPRESSOR_CAPEX_DOL_kgPD = NUM_DAYS*COMPRESSOR_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365 )
LIQUEFACTION_CAPEX_DOL_kgPD = NUM_DAYS*LIQUEFACTION_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
FUELCELL_CAPEX_DOL_kgPD = NUM_DAYS*FUELCELL_CAPEX_DOL_kgPD / (FACILITY_LIFETIME * 365)
print(LIQUEFACTION_CAPEX_DOL_kgPD)
print(ELECTROLYSIS_CAPEX_DOL_kgPD)
print(len(hourly_dayahead_dol_per_kwh))
print(hourly_dayahead_dol_per_kwh)

21.315068493150687
9.863013698630137
2400
0       0.02010
1       0.01962
2       0.01921
3       0.01879
4       0.02068
         ...   
2395    0.03264
2396    0.02714
2397    0.02412
2398    0.02154
2399    0.01596
Name: price_dayahead, Length: 2400, dtype: float64


In [124]:
ERCTO_CO2_kg_per_kwh = 0.42062
SOLAR_CAPACITY = 400

intial_state = {
        "grid_running_sum": 0,
        "h2_offtake_running_sum":0, 
        "realtime_price_$_per_kwh": 0,
        "realtime_production_kwh":0,
        "realtime_supplied_kwh": 0,
        "solar_production_kwh": 0, 
        "electrolyser_consumption_kwh": 0, 
        "compressor_consumption_kwh": 0, 
        "liquefaction_consumption_kwh": 0,
        "electrolyser_produced_kg": 0, 
        "compressor_produced_kg": 0, 
        "liquefaction_produced_kg": 0,
        "compress_to_liquefaction": 0,
        "fuel_cell_consumed_kg":0,
        "gh2_storage_level_kg": 0,
        # TODO: if problem size allows, add in big-M constraint 
        "gh2_storage_net_inflow": 0,
        "gh2_storge_inflow_kg": 0, 
        "gh2_storage_outflow_kg": 0
    }
h = 24
t = 0 

# for i in [2.0,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
i = H2_SALES_PRICE_DOL_per_kg 
with open(os.path.join('data', 'results','deterministic', f'facility_summary_results_{i}.json'), 'r') as file:
        data = json.load(file)

prev_state_vector = intial_state

hourly_solar_production_kwh = hourly_dayahead_realtime_solar_lz['electricity']/hourly_dayahead_realtime_solar_lz['electricity'].max()
#   GRID 
#       CONVERT $/MWh -> $/kWh
hourly_dayahead_dol_per_kwh = hourly_dayahead_realtime_solar_lz['price_dayahead'] / 1000

result_list = [ ]
for t in range(24):
    if t % 200 == 0: 
        print(f"up to {t}")
    solar_consumed_facility = hourly_solar_production_kwh[t:t+h]
    hourly_dayahead_dol_per_kwh_h = hourly_dayahead_dol_per_kwh[t:t+h]
    T=h
    
    problem, operations, facility_summary, prev_state_vector = run_rolling_horizon_opt(
            SOLAR_CAPACITY,
            SOLAR_PPA_DOL_KWH,
            solar_consumed_facility,
            hourly_dayahead_dol_per_kwh_h,
            ELECTROLYSIS_EFFICIENCY,
            COMPRESSION_EFFICIENCY,
            LIQUEFACTION_EFFICIENCY,
            FUEL_CELL_EFFICIENCY,
            LIQUEFACTION_MAX_UP,
            LIQUEFACTION_MAX_DOWN,
            data['electrolyser_capacity_ph'],
            data['compressor_capacity_ph'],
            data['liquefaction_capacity_ph'],
            data['fuelcell_capacity_ph'],
            H2_SALES_PRICE_DOL_per_kg,
            GREEN_THRESHOLD_kg_per_kg,
            ERCTO_CO2_kg_per_kwh,
            T,
            prev_state_vector 
            )
    operations.to_csv(os.path.join('data', 'results','rolling_horizon', 'operations',f'operations_results_t{t}_{i}.csv'))
    
    # with open(os.path.join('data', 'results','rolling_horizon','facility_summary', f'facility_summary_results_t{t}_{i}.json'), 'w') as file:
    #     json.dump(facility_summary, file, indent=4)
    
    result_list.append(prev_state_vector)

up to 0
previous liquefaction 0
previous liquefaction 0.0


previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.5048430542777923
previous liquefaction 0.6731240723703897
previous liquefaction 0.8414050904629872
previous liquefaction 1.0096861085555846
previous liquefaction 1.177967126648182
previous liquefaction 1.2408483928477416
previous liquefaction 1.085682467337451
previous liquefaction 0.9174014492448536
previous liquefaction 0.7491204311522558
previous liquefaction 0.5808394130596584
previous liquefaction 0.4125583949670609
previous liquefaction 0.2442773768744635
previous liquefaction 0.07599635878186606
previous liquefaction 0.0
previous liquefaction 0.0
previous liquefaction 0.0


In [121]:
result_df = pd.DataFrame(result_list)
result_df['hourly_profit'] = result_df['hourly_dayahead_dol_per_kwh'] * (result_df['realtime_supplied_kwh'] - result_df['realtime_production_kwh']) + \
      result_df['liquefaction_produced_kg'] * H2_SALES_PRICE_DOL_per_kg - \
        result_df['solar_production_kwh']*SOLAR_PPA_DOL_KWH


In [122]:
result_df[['solar_available_kwh',\
           'liquefaction_produced_kg','liquefaction_operating', 'liquefaction_state_change',\
              'liquefaction_state_change_up', 'liquefaction_state_change_down', 'liquefaction_minimum_production']]

,solar_available_kwh,liquefaction_produced_kg,liquefaction_operating,liquefaction_state_change,liquefaction_state_change_up,liquefaction_state_change_down,liquefaction_minimum_production
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.504843
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.504843
2,0.000000,0.000000,0.0,-0.0,0.0,0.0,0.504843
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.504843
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.504843
5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.504843
6,3.171008,0.000000,0.0,0.0,0.0,0.0,0.504843
7,38.052095,0.504843,1.0,1.0,1.0,0.0,0.504843
8,208.380521,0.673124,1.0,0.0,0.0,0.0,0.504843
9,272.706682,0.841405,1.0,0.0,0.0,0.0,0.504843


In [6]:
sum(result_df['hourly_profit']) * 4 * 1000 * FACILITY_LIFETIME
sum(result_df['liquefaction_produced_kg'])

NameError: name 'result_df' is not defined

In [3]:
for hydrogen_sales_price in [2.0,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]:
    result_df['hourly_profit'] = result_df['hourly_dayahead_dol_per_kwh'] * (result_df['realtime_supplied_kwh'] - result_df['realtime_production_kwh']) + \
      result_df['liquefaction_produced_kg'] * hydrogen_sales_price - \
        result_df['solar_production_kwh']*SOLAR_PPA_DOL_KWH
    
    lifetime_profits = sum(result_df['hourly_profit']) * 4 * 1000 * FACILITY_LIFETIME  - data['electrolyser_capacity_ph'] *  24 * ELECTROLYSIS_CAPEX_DOL_TPD - data['compressor_capacity_ph'] *  24 * COMPRESSOR_CAPEX_DOL_TPD - data['liquefaction_capacity_ph'] *  24 * LIQUEFACTION_CAPEX_DOL_TPD
    annual_profit = lifetime_profits / FACILITY_LIFETIME
    print(f"{hydrogen_sales_price} & {annual_profit} & {round(lifetime_profits,2)}\\\\")

NameError: name 'result_df' is not defined

In [5]:
for i in [2.0,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]:
        with open(os.path.join('data', 'results','deterministic', f'facility_summary_results_{i}.json'), 'r') as file:
                data = json.load(file)

        print(f"{round(data['electrolyser_capacity_ph'],2)} & {round(data['compressor_capacity_ph'],2)} & {round(data['liquefaction_capacity_ph'],2)}  & {round(data['fuelcell_capacity_ph'],2)}\\\\")

0.0 & 0.0 & 0.0  & 0.0\\
0.0 & 0.0 & 0.0  & 0.0\\
0.0 & 0.0 & 0.0  & 0.0\\
0.0 & 0.0 & 0.0  & 0.0\\
0.28 & 0.28 & 0.31  & 0.0\\
1.59 & 1.59 & 1.68  & 0.0\\
3.11 & 3.11 & 2.9  & 0.0\\
4.05 & 4.05 & 2.86  & 0.0\\
4.57 & 4.57 & 3.75  & 0.0\\
5.51 & 5.51 & 4.15  & 0.0\\
5.51 & 5.51 & 4.15  & 0.0\\


In [30]:
# assert that storage is operating correctly 
print(sum(result_df['compressor_produced_kg'] - result_df['liquefaction_produced_kg'] - result_df['gh2_storage_net_inflow']))
result_df.to_csv("operations_slack_test.csv")


1.6653345369377348e-16
